<p align="center">
<img src="Images/sorbonne_logo.png" alt="Logo" width="300"/>
</p>

# **PANEL DATA ECONOMETRICS - Code File Part 2**

* **Author**: Elia Landini
* **Student ID**: 12310239
* **Course**: EESM2-Financial Economics 
* **Class**: Panel Data Econometrics
* **Supervisor**: Jean-Bernard Chatelain 
* **Reference Paper**: Lofaro, A., & Di Bucchianico, S. (2025). Impact of monetary policy on functional income distribution: A panel vector autoregressive analysis. Economic Modelling, 107227
* **Reference Repository**: https://github.com/EliaLand/PVAR_MonetaryPolicy_FunctionalIncome

### **0) REQUIREMENTS SET-UP**

In [2]:
# Requirements.txt file installation
# !pip install -r requirements.txt

In [3]:
# Libraries import
import warnings
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import levene
from scipy.stats import ks_2samp
from scipy.stats import kstest
from scipy.stats import pearsonr
from scipy.stats import jarque_bera
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.nonparametric.smoothers_lowess import lowess
import sklearn.tree
import sklearn.metrics
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing 
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (roc_auc_score, roc_curve, confusion_matrix,
                             precision_score, recall_score, f1_score,
                             accuracy_score, precision_recall_curve, auc, 
                             RocCurveDisplay, ConfusionMatrixDisplay)
from sklearn.linear_model import (LinearRegression, LogisticRegression)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight
import plotly.express as px
import openpyxl as pxl
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
from IPython.display import Image
import itertools
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from ann_visualizer.visualize import ann_viz
from collections import Counter
import shap

In [4]:
# Statistical Significance labelling 
def significance_stars(p):
    if p < 0.001:
        return "***"  
    elif p < 0.01:
        return "**"    
    elif p < 0.05:
        return "*"   
    else:
        return ""

In [5]:
# We supress potential warnings with this command
warnings.filterwarnings("ignore")

In [6]:
# Data loading from CodeFile Part 1 for raw data and transformed variables
final_trans_df = pd.read_csv("Data/final_trans_df.csv")
final_trans_df

,year,country,i,P,W,WR,GDP,LS,PCOM,UN,...,WR_TWFE,GDP_TWFE,LS_TWFE,PCOM_TWFE,UN_TWFE,SHORTUN_TWFE,LONGUN_TWFE,LF_TWFE,REER_TWFE,SH_TWFE
0,1970-01-01,Australia,7.108333,9.1,6.413648,76.120158,2.835193e+05,59.7,2.113115,1.653364,...,6.652899,6.417193e+05,-1.599571,1.421085e-14,0.083200,NaN,NaN,3746.924114,2.873095,-2.544304
1,1971-01-01,Australia,7.008333,9.6,6.909308,77.071751,2.946057e+05,60.4,2.709369,1.907799,...,6.815177,6.194218e+05,-1.535286,2.131628e-14,-0.010331,NaN,NaN,3673.600806,5.056312,-1.057045
2,1972-01-01,Australia,5.083333,10.2,7.284607,75.808223,3.023225e+05,60.3,2.925285,2.610752,...,5.360967,5.765079e+05,-1.728143,0.000000e+00,0.306079,NaN,NaN,3516.775114,4.881667,-2.286273
3,1973-01-01,Australia,6.983333,11.1,8.096658,74.465333,3.147389e+05,60.8,4.174010,2.309362,...,0.878244,5.265239e+05,-0.949571,0.000000e+00,0.280121,NaN,NaN,3252.168575,15.521260,-2.962910
4,1974-01-01,Australia,13.158333,12.9,9.688749,75.817692,3.189447e+05,66.9,13.802962,2.670259,...,2.643711,5.213183e+05,3.743286,1.421085e-14,0.703663,NaN,NaN,2769.733122,17.166315,0.476242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2015-01-01,USA,0.226667,100.0,65.600000,62.655205,1.820602e+07,56.8,66.099106,5.279920,...,9.355536,5.625355e+06,0.469433,0.000000e+00,-2.158072,-366.715087,510.316135,23419.420004,12.133839,1.115529
746,2016-01-01,USA,0.644167,101.3,66.400000,62.819300,1.850960e+07,56.8,56.023884,4.869110,...,8.342223,5.882897e+06,0.702767,1.776357e-15,-2.012960,-557.504449,282.317657,25183.856004,14.946635,3.544706
747,2017-01-01,USA,1.152500,103.4,68.600000,63.695450,1.892457e+07,56.9,69.357989,4.355201,...,8.851479,6.232285e+06,1.196100,-1.065814e-14,-1.792373,-943.479016,65.509990,26116.245337,14.351430,4.758716
748,2018-01-01,USA,2.188333,105.9,70.900000,64.279238,1.948198e+07,56.8,89.358138,3.895676,...,8.792271,6.728788e+06,0.876100,-1.421085e-14,-1.493159,-1224.491436,-171.593610,27580.072337,12.878166,5.804448


### **2) PART 2 - CLASSIC BENCHMARK MULTIVARIATE PANEL DATA ESTIMATORS**

##### <span style="color: orange"> **2.20) In a single table, report and comment the results of estimations of Between, Within (one-way fixed effects, (fe)) and Mundlak (random effects (re) including all X(i.) as regressors), two-way fixed effects (add year dummies in fe regression) and First differences, including all explanatory variables except the ones with high near-multicollinearity after their transformation.** </span>

##### <span style="color: orange"> **2.21) If, for the first differences dependent variable, it remains a simple auto-correlation above 0.1, a dynamic panel estimator can be tried. The estimators of the generalized method of moments (GMM) for panel data are only valid for short time panel T<10 and they face the issue of too many weak instruments. We suggest using its precursor, the Anderson-Hsiao (1981) estimator which allows to check the first stage of instrumental variables and to test for weak lagged instruments. Estimate an auto-regressive distributed lag (ARDL) model for dynamic panel data including the first lag of the dependent variable (for example: GDP per head growth) and the first lag of the key explanatory variable (for example: foreign aid/GDP), adding the first lag of other control variables is optional: Δ GDPGi,t = βy Δ GDPGi,t-1 + β1 Δ (aid/GDP)i,t + β2 Δ (aid/GDP)i,t-1 + Δ Controls i,t + Δ αi + Δ αt + Δ εi** </span>

### **2) PART 2B - OPTIONAL (if one of your variable is time-invariant z(i))**

[Unfortunately,  Lofaro, A., & Di Bucchianico, S. (2025) present PVAR model including 14 variables, country (i) and year (t) dependent, with only one exception for Energy Commodities Price Index, which is country-invariant, and hence only time dependent.]

##### <span style="color: orange"> **2.22) If one of your variable is time-invariant z(i), run a baseline Hausman Taylor estimation (pre-coded only in STATA) including all X(i.) as instruments. Comment the results. Else skip this question.** </span>

[n.a.]

##### <span style="color: orange"> **2.23) If one of your variable is time-invariant z(i), run a between regression on z(i) explained by X(i.) and other time invariant variable (only with N observations). If the R2 is low, this may signal X(i.) are weak instruments poorly correlated with the variable z(i) to be instrumented. Comment. Else skip this question.** </span>

[n.a.]

##### <span style="color: orange"> **2.24) If one of your variable is time-invariant z(i), as seen above, time invariant explanatory variables cannot explain the time varying within variance of the dependent variable and the Hausman Taylor internal instruments estimator is not so practical. Therefore, a practical shortcut is to include a time invariant variable multiplied by a time varying variable (interaction term): z(i) multiplied for x(it). Generate such a variable Include this product AND foreign aid into a one way fixed effects regression. Plot the estimated marginal effect (derivative) with respect to ICRG as a function of EDA/GDP (which is positive and goes as far as 20%).** </span>

[n.a.]